# scraping한 rawdata 전처리 - review
- 형식에 맞게 클리닝

In [205]:
import pandas as pd

## review 데이터 클리닝 (진행 중)
1. review 데이터 합치기
2. 클리닝
    - column별 type 맞추기
    - user_id: id / 예매자 여부 분할
    - view_count: 조회 없애기 (숫자만 남기기)
    - like_count: 공감 ~ \n공감하기 없애기 (숫자만 남기기)
    - 베스트 리뷰 삭제 (중복임)
    - 라이브 리뷰라는 표시 삭제
    - title, text
        - 특수기호 처리
        - 토큰화
        - 등등등

1. review 데이터 합치기

In [181]:
import os

def concat_review_data():
    folder_path = "../data/rawdata/"
    infos = []

    for file_path in os.listdir(folder_path):
        if "review" in file_path:
            info = pd.read_csv(folder_path + file_path, converters={"performance_id" : str})
            print(f"[{file_path}]")
            print(info.info())
            infos.append(info)

    df = pd.concat(infos, ignore_index=True)
    df.to_csv(f"../data/preprocessed_data/total_review.csv", index=False, encoding='utf-8')

    return df

In [182]:
review = concat_review_data()

[뮤지컬_레베카_review.csv]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2614 entries, 0 to 2613
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  2614 non-null   object
 1   review_id       2614 non-null   int64 
 2   rating          2614 non-null   int64 
 3   user_id         2614 non-null   object
 4   date            2614 non-null   object
 5   view_count      2614 non-null   object
 6   like_count      2614 non-null   object
 7   title           2614 non-null   object
 8   text            2614 non-null   object
dtypes: int64(2), object(7)
memory usage: 183.9+ KB
None
[뮤지컬_브론테_review.csv]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  1445 non-null   object
 1   review_id       1445 non-null   int64 
 2   rating          1445 non

In [183]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38655 entries, 0 to 38654
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  38655 non-null  object
 1   review_id       38655 non-null  int64 
 2   rating          38655 non-null  int64 
 3   user_id         38655 non-null  object
 4   date            38655 non-null  object
 5   view_count      38655 non-null  object
 6   like_count      38655 non-null  object
 7   title           38642 non-null  object
 8   text            38654 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.7+ MB


In [184]:
review.head()

,performance_id,review_id,rating,user_id,date,view_count,like_count,title,text
0,23008837,0,5,tmdfl0***예매자,2024.02.05,조회 6,공감 0\n공감하기,추천합니다,최고의극
1,23008837,1,5,youi1***예매자,2024.01.02,조회 22,공감 0\n공감하기,레베카 공연 최고!!!!!!,배우들의 연기와 감정 최고였음!!!! 앙코르 기념공연 기대되요!!!
2,23008837,2,5,wani***예매자,2023.12.18,조회 13,공감 0\n공감하기,역시 레베카다,최애 공연입니다. 볼때마다 레전드 갱신!
3,23008837,3,5,tsk***예매자,2023.12.16,조회 48,공감 0\n공감하기,옥주연님의 덴버스 부인을 보고,5년 전? 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...
4,23008837,4,5,kys95***예매자,2023.12.13,조회 20,공감 0\n공감하기,믿고 보는 옥댄버!!!,물개박수 치면서 관람 했어요!!!\n정말 인생 뮤지컬이었던:)\n옥댄버 성량이 블루...


In [185]:
# 조회수 숫자형으로
review["view_count"] = review["view_count"].apply(lambda x: int(x.split(" ")[1]))

# 공감수 숫자형으로
review["like_count"] = review["like_count"].apply(lambda x: int(x[3:-5]))

In [186]:
# id에 나타나는 예매자 여부 삭제
review['user_id'] = review['user_id'].str.split("\*\*\*", expand=True)[0] + "***"

- 베스트 리뷰 제거

In [187]:
# 베스트 리뷰로 등록되지 않았음에도 중복 리뷰가 있을 경우를 위해 title까지 포함해 1차 중복 제거 진행
# title 제외(베스트 키워드가 포함되므로)한 뒤 중복 제거 베스트 리뷰가 무조건 빠르므로 keep="last"
review.drop_duplicates(subset=["performance_id", "user_id", "date", "view_count", "like_count", "title", "text"], keep='first', inplace=True, ignore_index=True)
review.drop_duplicates(subset=["performance_id", "user_id", "date", "view_count", "like_count", "text"], keep='last', inplace=True, ignore_index=True)

In [188]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24854 entries, 0 to 24853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24854 non-null  object
 1   review_id       24854 non-null  int64 
 2   rating          24854 non-null  int64 
 3   user_id         24854 non-null  object
 4   date            24854 non-null  object
 5   view_count      24854 non-null  int64 
 6   like_count      24854 non-null  int64 
 7   title           24841 non-null  object
 8   text            24853 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


- title, text Null(결측치) 값 ''으로 치환
    - title, text에만 결측치 존재 -> df.fillna('')

In [189]:
# Null 값 확인
review[review["title"].isnull() | review["text"].isnull()]

,performance_id,review_id,rating,user_id,date,view_count,like_count,title,text
5799,24001224,647,5,wjdfksdl3***,2024.04.08,1,0,NaN,초연때 처음 봤던 캐스팅으로 다시 봤는데 정말 너무너무너무너무 좋았어요ㅠㅠ 오즈가 ...
7766,24001224,2685,5,wjdfksdl3***,2024.03.04,1,0,NaN,다시 돌아온 오즈!! 넘버들도 너무 좋고 너무 재밌어요ㅠㅠ 이번 시즌은 꼭 OST가...
8226,24002031,313,5,wjdfksdl3***,2024.04.15,0,0,NaN,항상 페어마다 다른 재미가 있어서 너무 좋아요~
8256,24002031,345,5,wjdfksdl3***,2024.04.15,1,0,NaN,웨스턴 스토리 넘버가 너무 좋아요bb 이번 시즌 전캐전곡 OST 꼭 나왔으면 좋겠습...
8765,24002031,866,5,wjdfksdl3***,2024.04.08,1,0,NaN,페어마다 다른 매력을 가지고 있어서 볼 때마다 새로운 공연을 보는 것 같아요! 너무...
8784,24002031,886,5,wjdfksdl3***,2024.04.08,0,0,NaN,"아 정말 웃다가 울고 나왔어요ㅠㅠㅋㅋㅋㅋㅋㅋㅋ 와이어트의 화려한 골반 너무 잘봤고,..."
9291,24002031,1421,5,wjdfksdl3***,2024.04.01,1,0,NaN,너무 웃겨서 웃다가 눈물났어요ㅋㅋㅋㅋㅋㅋ 찐찐이와 원종''환'' 정말 최고!!!
9737,24002031,1887,5,wjdfksdl3***,2024.03.25,2,0,NaN,와이어트의 골반춤이 자꾸 떠올라요ㅠㅋㅋㅋㅋ 같이 골반을 돌려야할 것 같은데.. OS...
9785,24002031,1937,5,wjdfksdl3***,2024.03.25,2,0,NaN,웃다가 배꼽 빠질뻔했어요ㅋㅋㅋㅋㅋㅋㅋ 배우분들 다 너무 노래도 잘하시고 웃기시고 최고bb
10198,24002031,2358,5,wjdfksdl3***,2024.03.18,4,0,NaN,어떤 캐스팅으로 봐도 너무 재밌는 웨스턴 스토리! 이번 시즌엔 꼭 전캐전곡 OST가...


In [190]:
# title, text 결측치 채우기
review.fillna('', inplace=True)

In [191]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24854 entries, 0 to 24853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24854 non-null  object
 1   review_id       24854 non-null  int64 
 2   rating          24854 non-null  int64 
 3   user_id         24854 non-null  object
 4   date            24854 non-null  object
 5   view_count      24854 non-null  int64 
 6   like_count      24854 non-null  int64 
 7   title           24854 non-null  object
 8   text            24854 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


- title 말미에 붙는 "\n라이브 후기" 제거
    - title에 라이브 후기가 아님에도 \n이 있는 경우는 없다
    - 제목이 없는데 라이브 후기인 경우 -> "\n라이브 후기" -> "" 형태가 되도록

In [196]:
review["title"] = review["title"].apply(lambda x : x.split("\n")[0]).replace("라이브 후기", "")

- column 이름 (field 이름) 통일

In [202]:
review.columns

Index(['performance_id', 'review_id', 'rating', 'user_id', 'date',
       'view_count', 'like_count', 'title', 'text'],
      dtype='object')

In [203]:
review.rename(columns={"review_id" : "review_num", "text" : "content"}, inplace=True)

In [204]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24854 entries, 0 to 24853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   performance_id  24854 non-null  object
 1   review_num      24854 non-null  int64 
 2   rating          24854 non-null  int64 
 3   user_id         24854 non-null  object
 4   date            24854 non-null  object
 5   view_count      24854 non-null  int64 
 6   like_count      24854 non-null  int64 
 7   title           24854 non-null  object
 8   content         24854 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


---

In [17]:
review['title'] = review['title'].str.replace('[^가-힣]', ' ', regex = True)
review['text'] = review['text'].str.replace('[^가-힣]', ' ', regex = True)

review[['title', 'text']].head()

,title,text
0,추천합니다,최고의극
1,레베카 공연 최고,배우들의 연기와 감정 최고였음 앙코르 기념공연 기대되요
2,역시 레베카다,최애 공연입니다 볼때마다 레전드 갱신
3,옥주연님의 덴버스 부인을 보고,년 전 신영숙님의 덴버스 부인을 보고 강한 충격을 먹고 옥주연님의 연기도 너무 ...
4,믿고 보는 옥댄버,물개박수 치면서 관람 했어요 정말 인생 뮤지컬이었던 옥댄버 성량이 블루스퀘...


In [18]:
all_title = ' '.join(review['title'].values.astype(str))
all_text = ' '.join(review['text'].values.astype(str))

all_content = all_title + all_text

In [49]:
# 문장에서 명사를 추출하는 형태소 분석 라이브러리
from konlpy.tag import Hannanum

# Hannanum 객체를 생성한 후, .nouns()를 통해 명사를 추출합니다.

hannanum = Hannanum()
nouns = hannanum.nouns(all_content)
words = [noun for noun in nouns if len(noun) > 1]

In [19]:
# 시각화에 쓰이는 라이브러리
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# 횟수를 기반으로 딕셔너리 생성
from collections import Counter

In [ ]:
# WordCloud를 이용해 텍스트 구름을 만들어봅시다.

wordcloud = WordCloud(
    font_path = "C:/Windows/Fonts/malgun.ttf",
    background_color="white",
    width = 1000,
    height = 1000
)

img = wordcloud.generate_from_frequencies(counter)
plt.imshow(img)